# TMA Analysis code supporting technote SITCOMTN-057
Craig Lage - 15-Nov-23  Updated 18-Dec-23 to use TMAEvents.

This notebook characterizes several things associated with the TMA: 

1. Slew and Settle times
2. Mount jitter


# Prepare the notebook

In [ ]:
# Directory to store the data
from pathlib import Path
data_dir = Path("./plots")
data_dir.mkdir(exist_ok=True, parents=True)

# You can include a list of different days and blocks,
# including more than one block on the same day, if desired
# You can also create a loop to build this list.
day_block_pairs = [[20231214, 146], [20231215, 146]]

# For the jitter tests, the parameters below allow you to add
# a delay after the start of the tracking event, or before the
# end of the tracking event
delay_after_start = 0.0
delay_before_end = 5.0

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import getEfdData

In [ ]:
client = EfdClient("usdf_efd")
eventMaker = TMAEventMaker()

In [ ]:
def tickFormatter(value, tick_number):
    # Convert the value to a string without subtracting large numbers                                                 
    # tick_number is unused.                                                                                          
    return f"{value:.2f}"


# Cycle through the events plotting the tracking errors, and getting the data for the slew time histogram.

In [ ]:
# These are for plotting the slew time distributions
slew_times = []
slew_dist = []

pdf = PdfPages(str(data_dir / "Mount_Jitter_Plots.pdf"))
fig = plt.figure(figsize=(10, 8))
for i, [dayObs, blockNum] in enumerate(day_block_pairs):
    events = eventMaker.getEvents(dayObs)
    blockParser = BlockParser(dayObs)  # get the info for the day
    seqNums = blockParser.getSeqNums(blockNum)  # get the seqNums for the specified block
    Nslews = 0
    Ntracks = 0

    for event in events:
        if event.type.name == 'TRACKING':
            Ntracks += 1
        elif event.type.name == 'SLEWING':
            Nslews += 1
    print(f"There are {len(events)} events, {Nslews} slewing and {Ntracks} tracking")    
    print(f"On {dayObs}, there are {len(seqNums)} events associated with block {blockNum}")
    if i == 0:
        # for making the example slew plot
        medianSeqNum = int((seqNums[0] + seqNums[-1]) / 2)
        firstEvent = medianSeqNum - 4
        lastEvent = medianSeqNum + 4
    if i >0:
        continue

    for seqNum in seqNums[120:125]:
            #try:
            event = events[seqNum]
            if event.type.name == 'SLEWING':
                start = event.begin
                end = event.end
            elif event.type.name == 'TRACKING':
                start = event.begin + TimeDelta(delay_after_start, format='sec')
                end = event.end - TimeDelta(delay_before_end, format='sec')
            azimuthData = getEfdData(client,'lsst.sal.MTMount.azimuth',
            event=event)
            elevationData = getEfdData(client,'lsst.sal.MTMount.elevation',
            event=event)
            print(seqNum, event.begin.isot, event.type.name)
            az_times = azimuthData['timestamp'].values
            el_times = elevationData['timestamp'].values
            az_values = azimuthData['actualPosition'].values
            el_values = elevationData['actualPosition'].values
    
            if event.type.name == 'SLEWING':
                continue
                az_shift = abs(az_values[0] - az_values[-1])
                el_shift = abs(el_values[0] - el_values[-1])
                az_shift_mod = az_shift * np.cos(el_values[0]*np.pi/180.0)
                shift = np.sqrt(el_shift*el_shift + az_shift_mod*az_shift_mod)
                if shift > 0.2 and shift < 10.0:
                    slew_dist.append(shift)
                    slew_times.append(event.duration)

            elif event.type.name == 'TRACKING':
                # Need to pad this data for the interpolation to work right                                                       
                pointingData = getEfdData(client, 'lsst.sal.MTPtg.currentTargetStatus',
                                          event=event, prePadding=1.0, postPadding=0.0)
                ptg_times = pointingData['timestamp'].values
                # Need to interpolate because demand and actual data streams                                                      
                # have different lengths                                                                                          
                az_demand_interp = np.interp(az_times, ptg_times, pointingData['demandAz'])
                el_demand_interp = np.interp(el_times, ptg_times, pointingData['demandEl'])
                az_error = (az_values - az_demand_interp) * 3600
                el_error = (el_values - el_demand_interp) * 3600
                # Because of small timebase errors, there can be an offset in the                                                 
                # errors. I take this out by subtracting the median of the errors.                                                
                # This is a fudge, but I think better than the polynomial fit.                                                    
                az_error -= np.median(az_error)
                el_error -= np.median(el_error)
                azimuthData['azError'] = az_error
                elevationData['elError'] = el_error

                ax1, ax1p5, ax2 = fig.subplots(3, sharex=True,
                                       gridspec_kw={'wspace': 0,
                                                    'hspace': 0,
                                                    'height_ratios': [2.5, 1, 1]})

                # Use the native color cycle for the lines. Because they're on different                                              
                # axes they don't cycle by themselves                                                                                 
                lineColors = [p['color'] for p in plt.rcParams['axes.prop_cycle']]
                colorCounter = 0
            
                ax1.plot(azimuthData['actualPosition'], label='Azimuth position', 
                         c=lineColors[colorCounter])
                colorCounter += 1
                ax1.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
                ax1.set_ylabel('Azimuth (degrees)')
            
                ax1_twin = ax1.twinx()
                ax1_twin.plot(elevationData['actualPosition'], label='Elevation position', 
                              c=lineColors[colorCounter])
                colorCounter += 1
                ax1_twin.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
                ax1_twin.set_ylabel('Elevation (degrees)')
                ax1.set_xticks([])  # remove x tick labels on the hidden upper x-axis                                                 
                az_rms = np.sqrt(np.mean(az_error * az_error))
                el_rms = np.sqrt(np.mean(el_error * el_error))
        
                # Calculate Image impact RMS                                                                                      
                # We are less sensitive to Az errors near the zenith                                                              
                image_az_rms = az_rms * np.cos(el_values[0] * np.pi / 180.0)
                image_el_rms = el_rms
                print(seqNum, image_el_rms, image_az_rms)
                ax1p5.plot(azimuthData['azError'], label='Azimuth error', 
                           c=lineColors[colorCounter])
                colorCounter += 1
                ax1p5.plot(elevationData['elError'], label='Elevation error', 
                           c=lineColors[colorCounter])
                colorCounter += 1
                ax1p5.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
                ax1p5.set_ylabel('Tracking error (arcsec)')
                ax1p5.set_xticks([])  # remove x tick labels on the hidden upper x-axis                                           
                ax1p5.set_ylim(-0.5, 0.5)
                ax1p5.set_yticks([-0.25, 0.0, 0.25])
                ax1p5.legend()
                ax1p5.text(0.2, 0.9,
                           f'Az image RMS = {image_az_rms:.3f} arcsec,   El image RMS = {image_el_rms:.3f} arcsec',
                           transform=ax1p5.transAxes)
                ax2_twin = ax2.twinx()
                ax2.plot(azimuthData['actualTorque'], label='Azimuth torque', 
                         c=lineColors[colorCounter])
                colorCounter += 1
                ax2_twin.plot(elevationData['actualTorque'], label='Elevation torque', 
                              c=lineColors[colorCounter])
                colorCounter += 1
                ax2.set_ylabel('Azimuth torque (Nm)')
                ax2_twin.set_ylabel('Elevation torque (Nm)')
                ax2.set_xlabel('Time (UTC)')  
                # yes, it really is UTC, matplotlib converts this automatically!                        
            
                # put the ticks at an angle, and right align with the tick marks                                                      
                ax2.set_xticks(ax2.get_xticks())  # needed to supress a user warning                                                  
                xlabels = ax2.get_xticks()
                ax2.set_xticklabels(xlabels, rotation=40, ha='right')
                ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
                ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                print("Saving this figure")
                pdf.savefig(fig)  # saves the current figure into a pdf page
                plt.clf()


    
            
            else:
                continue
        
        
            #except:
            #continue

pdf.close()

In [ ]:
plt.plot(azimuthData['actualPosition'])

In [ ]:
plt.plot(azimuthData['actualTorque'])

In [ ]:
plt.plot(az_times, az_values)
#plt.plot(az_demand, ls='--')
plt.plot(pointingData['timestamp'], pointingData['demandAz'] )

In [ ]:
pointingData['timestamp'][-1]

In [ ]:
azimuthData['timestamp'][-1]

In [ ]:
%matplotlib inline
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.hist(slew_times, bins=50, range=(0,50))
plt.xlabel("Slew and settle time (seconds)")
plt.xlim(0.0, 10.0)
plt.subplot(1,2,2)
plt.scatter(slew_dist, slew_times)
plt.ylabel("Slew and settle time(sec)")
plt.xlabel("Slew distance (degrees)")
plt.plot([3.5,3.5],[0,10], ls='--', color='black')
plt.plot([0,10],[4.0,4.0], ls='--', color='black')
plt.xlim(0,10)
plt.ylim(0,10)
#plt.savefig(str(data_dir / "Slew_Settle_Times_Corrected.pdf"))

# This is just to plot an example of the slews and tracks

In [ ]:
[dayObs, blockNum] = day_block_pairs[0]
events = eventMaker.getEvents(dayObs)
blockParser = BlockParser(dayObs)  # get the info for the day
seqNums = blockParser.getSeqNums(blockNum)  # get the seqNums for the specified block
print(f"On {dayObs}, there are {len(seqNums)} events associated with block {blockNum}")
start = events[firstEvent].begin
end = events[lastEvent].end
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                        ['*'],  start, end)
plt.subplot(1,1,1)
plt.title("Azimuth Slew and Tracking")
ax1 = az['actualPosition'].plot(color='red')
for i in range(firstEvent, lastEvent+1):
    if events[i].type.name != 'SLEWING':
        continue
    ss_time = events[i].begin.isot  
    ax1.axvline(ss_time, color="black", linestyle="--")
    ip_time = events[i].end.isot  
    ax1.axvline(ip_time, color="blue", linestyle="--")
#ax1.set_xlim(start.isot, end.isot)
ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
ax1.set_ylabel("Azimuth(degrees)")
ax1.legend()
plt.savefig(str(data_dir / f"Slew_Track_Example_{dayObs}.pdf"))

# This plots the jitter plots for all of the tracks.

In [ ]:
# Plotting the mount plots
azRmsVals = []
elRmsVals = []
imRmsVals = []
fig = plt.figure(figsize = (8,8))
for index in range(len(inPos) - 1):

    plotAz = az[(az['timestamp'] > inPos[index]) & (az['timestamp'] < start_slew_times[index+1])]
    plotEl = el[(el['timestamp'] > inPos[index]) & (el['timestamp'] < start_slew_times[index+1])]
    ss_time = Time(start_slew_times[index+1], format='unix_tai', scale='utc').isot
    ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot


    # Calculate the tracking errors
    az_vals = np.array(plotAz.values[:,0])
    el_vals = np.array(plotEl.values[:,0])
    times_az = plotAz.values[:,1]
    times_el = plotEl.values[:,1]
    # The fits are much better if the time variable
    # is centered in the interval
    time_delta_az = times_az[int(len(plotAz.values) / 2)]
    time_delta_el = times_el[int(len(plotEl.values) / 2)]
    fit_times_az = [(times_az[i]-time_delta_az) for i in range(len(times_az))]
    fit_times_el = [(times_el[i]-time_delta_el) for i in range(len(times_el))]

    # Fit with a polynomial
    az_fit = np.polyfit(fit_times_az, az_vals, 4)
    el_fit = np.polyfit(fit_times_el, el_vals, 4)
    az_model = np.polyval(az_fit, fit_times_az)
    el_model = np.polyval(el_fit, fit_times_el)

    # Errors in arcseconds
    az_error = (az_vals - az_model) * 3600
    el_error = (el_vals - el_model) * 3600
    
    # Drive velocities
    az_vel = (az_model[-1] - az_model[0]) / (fit_times_az[-1] - fit_times_az[0]) * 3600.0
    el_vel = (el_model[-1] - el_model[0]) / (fit_times_el[-1] - fit_times_el[0]) * 3600.0
    
    # Calculate RMS
    az_rms = np.sqrt(np.mean(az_error * az_error))
    el_rms = np.sqrt(np.mean(el_error * el_error))
    azRmsVals.append(az_rms)
    elRmsVals.append(el_rms)
    # Calculate Image impact RMS
    # We are less sensitive to Az errors near the zenith
    image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
    image_el_rms = el_rms
    imRmsVals.append(np.sqrt(image_az_rms*image_az_rms + image_el_rms*image_el_rms))

    fig.clear()
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    plt.suptitle(f"MT Mount Jitter - {ip_time}", fontsize = 18)
    plt.subplot(2,2,1)
    ax1 = plotAz['actualPosition'].plot(legend=True, color='red')
    ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
    ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
    ax1.set_title(f"Azimuth\nAve velocity={az_vel:.1f} arcsec/sec")
    ax1.set_ylabel("Degrees")
    ax1.legend()
    plt.subplot(2,2,2)
    ax3 = plotEl['actualPosition'].plot(legend=True, color='green')
    ax3.axvline(ss_time, color="black", linestyle="--", label="Start slew")
    ax3.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
    ax3.set_title(f"Elevation\nAve velocity={el_vel:.1f} arcsec/sec")
    ax3.set_ylabel("Degrees")
    ax3.legend()
    plt.subplot(2,2,3)
    plt.plot(fit_times_az, az_error, color='red')
    plt.title(f"Azimuth RMS error = {az_rms:.3f} arcseconds\n"
              f"  Image RMS error = {image_az_rms:.3f} arcseconds", fontsize=10)
    plt.ylim(-0.2,0.2)
    plt.xticks([])
    plt.ylabel("ArcSeconds")
    plt.subplot(2,2,4)
    plt.plot(fit_times_el, el_error, color='green')
    plt.title(f"Elevation RMS error = {el_rms:.3f} arcseconds\n"
              f"  Image RMS error = {image_el_rms:.3f} arcseconds", fontsize=10)
    plt.ylim(-0.2,0.2)
    plt.xticks([])
    plt.ylabel("ArcSeconds")
    timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
    plt.savefig(str(data_dir / f"MT_Mount_Jitter_Corrected_{timestamp}.pdf"))


mount_data = {}
mount_data['start_slew_times'] = start_slew_times
mount_data['inPos'] = inPos
mount_data['slew_times'] = slew_times
mount_data['slew_dist'] = slew_dist
mount_data['azRmsVals'] = azRmsVals
mount_data['elRmsVals'] = elRmsVals
mount_data['imRmsVals'] = imRmsVals

filename = str(data_dir / "mount_data_corrected.pkl")
file = open(filename, 'wb')
pkl.dump(mount_data, file)
file.close()



# This allows you to unpickle the pickled data, since running all of the tracking jitters takes time.

In [ ]:
filename = str(data_dir / "mount_data.pkl")
file = open(filename, 'rb')
mount_data = pkl.load(file)
file.close()

# And finally, this plots histograms of the tracking jitter.

In [ ]:
fig = plt.figure(figsize=(16,8))
plt.suptitle("MT Mount RMS Jitter - 20220126", fontsize = 18)
azRmsVals = mount_data['azRmsVals']
elRmsVals = mount_data['elRmsVals']
imRmsVals = mount_data['imRmsVals']
azMed = np.median(azRmsVals)
elMed = np.median(elRmsVals)
imMed = np.median(imRmsVals)
plt.subplots_adjust(wspace=0.2)
plt.subplot(1,3,1)
plt.title(f"Azimuth RMS, N={len(azRmsVals)}")
plt.hist(azRmsVals, range=(0,0.2))
plt.text(0.1,120, f"Median={azMed:.3f}", fontsize=12)
plt.xlim(0,0.2)
plt.xlabel("RMS Jitter (arcseconds)")
plt.subplot(1,3,2)
plt.title(f"Elevation RMS, N={len(azRmsVals)}")
plt.hist(elRmsVals, range=(0,0.2))
plt.text(0.1,120, f"Median={elMed:.3f}", fontsize=12)
plt.xlim(0,0.2)
plt.xlabel("RMS Jitter (arcseconds)")
plt.subplot(1,3,3)
plt.title(f"Image Impact RMS, N={len(azRmsVals)}")
plt.hist(imRmsVals, range=(0,0.2))
plt.text(0.1,120, f"Median={imMed:.3f}", fontsize=12)
plt.xlim(0,0.2)
plt.xlabel("RMS Jitter (arcseconds)")
plt.savefig(str(data_dir / "Jitter_Summary_Corrected.pdf"))